# Bridge-T
Last update: 12 Feb 2024  

![Bridge T](Bridge-T.png){#fig-bridge-t}  



## Introduction

[electronics stackexchange](https://electronics.stackexchange.com/)

@fig-bridge-t

@lees2023

## Circuit description


## Circuit analysis

- Solve the equations for initial conditions on the caps and inductor.
- find the state equations

The net list for the filter is:
```
* T-Coil-ver1.asc
V1 1 0 1
L1 2 1 1 Rser=0
L2 3 2 1 Rser=0
L3 2 4 1 Rser=0
C1 5 0 1
C2 3 1 1
R1 3 0 1
R2 4 5 1
K1 L1 L2 0.5
.op
.backanno
.end
```

The following Python modules are used.

In [1]:
from sympy import *
import numpy as np
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import SymMNA
from IPython.display import display, Markdown, Math, Latex
init_printing()

In [2]:
t = symbols('t',positive=True)  # t > 0

### Load the netlist
The netlist generated by LTSpice is pasted into the cell below and some edits were made to remove the inductor series resistance and the independent sources are set to their DC values.

In [3]:
net_list = '''
V1 1 0 1
L1 2 1 1
L2 3 2 1 
L3 2 4 1
C1 5 0 1
C2 3 1 1
R1 3 0 1
R2 4 5 1
K1 L1 L2 0.5
'''

Generate the network equations.

In [4]:
report, network_df, df2, A, X, Z = SymMNA.smna(net_list)

# Put matricies into SymPy 
X = Matrix(X)
Z = Matrix(Z)

NE_sym = Eq(A*X,Z)

Generate markdown text to display the network equations.

In [5]:
temp = ''
for i in range(len(X)):
    temp += '${:s}$<br>'.format(latex(Eq((A*X)[i:i+1][0],Z[i])))

Markdown(temp)

$C_{2} s v_{1} - C_{2} s v_{3} - I_{L1} + I_{V1} = 0$<br>$I_{L1} - I_{L2} + I_{L3} = 0$<br>$- C_{2} s v_{1} + I_{L2} + v_{3} \left(C_{2} s + \frac{1}{R_{1}}\right) = 0$<br>$- I_{L3} + \frac{v_{4}}{R_{2}} - \frac{v_{5}}{R_{2}} = 0$<br>$v_{5} \left(C_{1} s + \frac{1}{R_{2}}\right) - \frac{v_{4}}{R_{2}} = 0$<br>$v_{1} = V_{1}$<br>$- I_{L1} L_{1} s - I_{L2} M_{1} s - v_{1} + v_{2} = 0$<br>$- I_{L1} M_{1} s - I_{L2} L_{2} s - v_{2} + v_{3} = 0$<br>$- I_{L3} L_{3} s + v_{2} - v_{4} = 0$<br>

As shown above MNA generated many equations and these would be difficult to solve by hand and a symbolic soultion would take a lot of computing time. The equations are displace in matrix notation.

In [6]:
NE_sym

⎡C₂⋅s⋅v₁ - C₂⋅s⋅v₃ - I_L1 + I_V1 ⎤       
⎢                                ⎥       
⎢       I_L1 - I_L2 + I_L3       ⎥       
⎢                                ⎥   ⎡0 ⎤
⎢                     ⎛       1 ⎞⎥   ⎢  ⎥
⎢-C₂⋅s⋅v₁ + I_L2 + v₃⋅⎜C₂⋅s + ──⎟⎥   ⎢0 ⎥
⎢                     ⎝       R₁⎠⎥   ⎢  ⎥
⎢                                ⎥   ⎢0 ⎥
⎢                v₄   v₅         ⎥   ⎢  ⎥
⎢        -I_L3 + ── - ──         ⎥   ⎢0 ⎥
⎢                R₂   R₂         ⎥   ⎢  ⎥
⎢                                ⎥ = ⎢0 ⎥
⎢         ⎛       1 ⎞   v₄       ⎥   ⎢  ⎥
⎢      v₅⋅⎜C₁⋅s + ──⎟ - ──       ⎥   ⎢V₁⎥
⎢         ⎝       R₂⎠   R₂       ⎥   ⎢  ⎥
⎢                                ⎥   ⎢0 ⎥
⎢               v₁               ⎥   ⎢  ⎥
⎢                                ⎥   ⎢0 ⎥
⎢-I_L1⋅L₁⋅s - I_L2⋅M₁⋅s - v₁ + v₂⎥   ⎢  ⎥
⎢                                ⎥   ⎣0 ⎦
⎢-I_L1⋅M₁⋅s - I_L2⋅L₂⋅s - v₂ + v₃⎥       
⎢                                ⎥       
⎣      -I_L3⋅L₃⋅s + v₂ - v₄      ⎦       

The symbols generated by the Python code are extraced by the SymPy function free_symbols and then declared as SymPy variables. 

In [7]:
# turn the free symbols into SymPy variables
var(str(NE_sym.free_symbols).replace('{','').replace('}',''))

Built a dictionary of element values.

In [8]:
element_values = SymMNA.get_part_values(network_df)
element_values

Generate the symbolic solution

In [9]:
U_sym = solve(NE_sym,X)

Display the symbolic solution

In [10]:
temp = ''
for i in U_sym.keys():
    temp += '${:s} = {:s}$<br>'.format(latex(i),latex(U_sym[i]))

Markdown(temp)

$v_{1} = V_{1}$<br>$v_{2} = \frac{C_{1} C_{2} L_{1} L_{3} R_{1} V_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} V_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} V_{1} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} V_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} L_{2} L_{3} V_{1} s^{3} + C_{1} L_{2} R_{2} V_{1} s^{2} + C_{1} L_{3} M_{1} V_{1} s^{3} + C_{1} L_{3} R_{1} V_{1} s^{2} + C_{1} M_{1} R_{2} V_{1} s^{2} + C_{1} R_{1} R_{2} V_{1} s + C_{2} L_{1} R_{1} V_{1} s^{2} + C_{2} L_{2} R_{1} V_{1} s^{2} + 2 C_{2} M_{1} R_{1} V_{1} s^{2} + L_{2} V_{1} s + M_{1} V_{1} s + R_{1} V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$v_{3} = \frac{C_{1} C_{2} L_{1} L_{2} R_{1} V_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} V_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} V_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} V_{1} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} V_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} V_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} L_{3} R_{1} V_{1} s^{2} - C_{1} M_{1} R_{1} V_{1} s^{2} + C_{1} R_{1} R_{2} V_{1} s + C_{2} L_{1} R_{1} V_{1} s^{2} + C_{2} L_{2} R_{1} V_{1} s^{2} + 2 C_{2} M_{1} R_{1} V_{1} s^{2} + R_{1} V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$v_{4} = \frac{C_{1} C_{2} L_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} C_{2} L_{2} R_{1} R_{2} V_{1} s^{3} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} V_{1} s^{3} + C_{1} L_{2} R_{2} V_{1} s^{2} + C_{1} M_{1} R_{2} V_{1} s^{2} + C_{1} R_{1} R_{2} V_{1} s + C_{2} L_{1} R_{1} V_{1} s^{2} + C_{2} L_{2} R_{1} V_{1} s^{2} + 2 C_{2} M_{1} R_{1} V_{1} s^{2} + L_{2} V_{1} s + M_{1} V_{1} s + R_{1} V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$v_{5} = \frac{C_{2} L_{1} R_{1} V_{1} s^{2} + C_{2} L_{2} R_{1} V_{1} s^{2} + 2 C_{2} M_{1} R_{1} V_{1} s^{2} + L_{2} V_{1} s + M_{1} V_{1} s + R_{1} V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$I_{V1} = \frac{- C_{1} C_{2} L_{1} L_{2} V_{1} s^{4} - C_{1} C_{2} L_{1} L_{3} V_{1} s^{4} - C_{1} C_{2} L_{1} R_{1} V_{1} s^{3} - C_{1} C_{2} L_{1} R_{2} V_{1} s^{3} - C_{1} C_{2} L_{2} L_{3} V_{1} s^{4} - C_{1} C_{2} L_{2} R_{1} V_{1} s^{3} - C_{1} C_{2} L_{2} R_{2} V_{1} s^{3} - 2 C_{1} C_{2} L_{3} M_{1} V_{1} s^{4} + C_{1} C_{2} M_{1}^{2} V_{1} s^{4} - 2 C_{1} C_{2} M_{1} R_{1} V_{1} s^{3} - 2 C_{1} C_{2} M_{1} R_{2} V_{1} s^{3} - C_{1} L_{2} V_{1} s^{2} - C_{1} L_{3} V_{1} s^{2} - C_{1} R_{1} V_{1} s - C_{1} R_{2} V_{1} s - C_{2} L_{1} V_{1} s^{2} - C_{2} L_{2} V_{1} s^{2} - 2 C_{2} M_{1} V_{1} s^{2} - V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$I_{L1} = \frac{- C_{1} C_{2} L_{2} R_{1} V_{1} s^{3} - C_{1} C_{2} M_{1} R_{1} V_{1} s^{3} - C_{1} L_{2} V_{1} s^{2} - C_{1} L_{3} V_{1} s^{2} - C_{1} R_{1} V_{1} s - C_{1} R_{2} V_{1} s - V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$I_{L2} = \frac{C_{1} C_{2} L_{1} R_{1} V_{1} s^{3} + C_{1} C_{2} M_{1} R_{1} V_{1} s^{3} - C_{1} L_{3} V_{1} s^{2} + C_{1} M_{1} V_{1} s^{2} - C_{1} R_{2} V_{1} s - V_{1}}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>$I_{L3} = \frac{C_{1} C_{2} L_{1} R_{1} V_{1} s^{3} + C_{1} C_{2} L_{2} R_{1} V_{1} s^{3} + 2 C_{1} C_{2} M_{1} R_{1} V_{1} s^{3} + C_{1} L_{2} V_{1} s^{2} + C_{1} M_{1} V_{1} s^{2} + C_{1} R_{1} V_{1} s}{C_{1} C_{2} L_{1} L_{2} R_{1} s^{4} + C_{1} C_{2} L_{1} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{1} R_{1} R_{2} s^{3} + C_{1} C_{2} L_{2} L_{3} R_{1} s^{4} + C_{1} C_{2} L_{2} R_{1} R_{2} s^{3} + 2 C_{1} C_{2} L_{3} M_{1} R_{1} s^{4} - C_{1} C_{2} M_{1}^{2} R_{1} s^{4} + 2 C_{1} C_{2} M_{1} R_{1} R_{2} s^{3} + C_{1} L_{1} L_{2} s^{3} + C_{1} L_{1} L_{3} s^{3} + C_{1} L_{1} R_{1} s^{2} + C_{1} L_{1} R_{2} s^{2} + C_{1} L_{2} L_{3} s^{3} + C_{1} L_{2} R_{2} s^{2} + 2 C_{1} L_{3} M_{1} s^{3} + C_{1} L_{3} R_{1} s^{2} - C_{1} M_{1}^{2} s^{3} + 2 C_{1} M_{1} R_{2} s^{2} + C_{1} R_{1} R_{2} s + C_{2} L_{1} R_{1} s^{2} + C_{2} L_{2} R_{1} s^{2} + 2 C_{2} M_{1} R_{1} s^{2} + L_{1} s + L_{2} s + 2 M_{1} s + R_{1}}$<br>